# Harmony EOSS API Tutorial - SWOT Test PI 19.4

## Before you start
Before you beginning this tutorial, make sure you have an account in the Earthdata Login UAT environment, which 
will be used for this notebook by visiting [https://uat.urs.earthdata.nasa.gov](https://uat.urs.earthdata.nasa.gov).
These accounts, as all Earthdata Login accounts, are free to create and only take a moment to set up.

## Set Up Authentication

We need some boilerplate up front to log in to Earthdata Login.  The function below will allow Python
scripts to log into any Earthdata Login application programmatically.  To avoid being prompted for
credentials every time you run and also allow clients such as curl to log in, you can add the following
to a `.netrc` (`_netrc` on Windows) file in your home directory:

```
machine uat.urs.earthdata.nasa.gov
    login <your username>
    password <your password>
```

Make sure that this file is only readable by the current user or you will receive an error stating
"netrc access too permissive."

`$ chmod 0600 ~/.netrc` 


In [ ]:
from urllib import request
from http.cookiejar import CookieJar
import getpass
import netrc

def setup_earthdata_login_auth(endpoint):
    """
    Set up the request library so that it authenticates against the given Earthdata Login
    endpoint and is able to track cookies between requests.  This looks in the .netrc file 
    first and if no credentials are found, it prompts for them.

    Valid endpoints include:
        uat.urs.earthdata.nasa.gov - Earthdata Login UAT (Harmony's current default)
        urs.earthdata.nasa.gov - Earthdata Login production
    """
    try:
        username, _, password = netrc.netrc().authenticators(endpoint)
    except (FileNotFoundError, TypeError):
        # FileNotFound = There's no .netrc file
        # TypeError = The endpoint isn't in the netrc file, causing the above to try unpacking None
        print('Please provide your Earthdata Login credentials to allow data access')
        print('Your credentials will only be passed to %s and will not be exposed in Jupyter' % (endpoint))
        username = input('Username:')
        password = getpass.getpass()

    manager = request.HTTPPasswordMgrWithDefaultRealm()
    manager.add_password(None, endpoint, username, password)
    auth = request.HTTPBasicAuthHandler(manager)

    jar = CookieJar()
    processor = request.HTTPCookieProcessor(jar)
    opener = request.build_opener(auth, processor)
    request.install_opener(opener)


Now call the above function to set up Earthdata Login for subsequent requests

In [ ]:
setup_earthdata_login_auth('uat.urs.earthdata.nasa.gov')

## Build the EOSS Root URL

Next we will build a URL for the EOSS service for a given granule.  To get data using the service, you need 
a [CMR UAT](https://cmr.uat.earthdata.nasa.gov) collection ID for a supported collection and the ID of a
granule within that collection.

By convention, all Harmony services are accessed through `<harmony_root>/<collection_id>/<service_name>`

In [ ]:
harmony_root = 'https://harmony.uat.earthdata.nasa.gov'
config = {
    'collection_id': 'C1233147319-EEDTEST',
    'eoss_version': '0.1.0'
}
eoss_collection_root = harmony_root+'/{collection_id}/eoss/{eoss_version}/items/'.format(**config)

print(eoss_collection_root)

## Reproject a Granule

We can now build onto the root URL in order to actually perform a transformation.  The first transformation is a reprojection of a selected granule. Coordinate reference systems are identified by a common name, EPSG code, or URI. Today, this is based on reference systems supported by gdal. Examples include: 'CRS:84', 'EPSG:32611'.

_At this time, this requires discovering the granule id from CMR_.  That information can then be appended to the root URL and used to call Harmony with the help of the `request` library.

Harmony stages transformed data in S3 to make it easy to do additional processing in the cloud. The response that Harmony returns is actually a redirect to the S3 location where your data is staged.  Should you call Harmony in a tool that follows redirects, like your web browser, your file will be seamlessly downloaded locally for you.  However, should you desire to do additional processing in AWS, you have that option as well by simply looking at the redirected URL.  The code snippet below uses "geturl()" to show the URL of your staged data.

## Reproject to Geographic

In [ ]:
reprojectionConfig = {
    'granule_id' : 'G1233147320-EEDTEST',
    'crs' : 'EPSG:4326'
}
eoss_reprojection_url = eoss_collection_root+'{granule_id}?crs={crs}'.format(**reprojectionConfig)


print('Request URL', eoss_reprojection_url)

with request.urlopen(eoss_reprojection_url) as response:
    print('URL for data staged in S3:', response.geturl())


In [ ]:
#+proj=longlat
reprojectionConfig = {
    'granule_id' : 'G1233147320-EEDTEST',
    'crs' : '%2Bproj%3Dlonglat'
}
eoss_reprojection_url = eoss_collection_root+'{granule_id}?crs={crs}'.format(**reprojectionConfig)


print('Request URL', eoss_reprojection_url)
print('')

with request.urlopen(eoss_reprojection_url) as response:
    print('URL for data staged in S3:', response.geturl())

## Reproject to Cylindrical Equal Area 

The third transformation is a reprojection of the data. This can be combined with the requests we already built above by simply specifying a coordinate reference system.  Coordinate reference systems are identified by a common name, EPSG code, or URI. Today, this is based on reference systems supported by gdal. Examples include: 'CRS:84', 'EPSG:32611'.

In [ ]:
reprojectionConfig = {
    'granule_id' : 'G1233147320-EEDTEST',
    'crs' : '%2Bproj%3Dcea'
}
eoss_reprojection_url = eoss_collection_root+'{granule_id}?crs={crs}'.format(**reprojectionConfig)


print('Request URL', eoss_reprojection_url)

with request.urlopen(eoss_reprojection_url) as response:
    print('URL for data staged in S3:', response.geturl())

In [ ]:
reprojectionConfig = {
    'granule_id' : 'G1233147320-EEDTEST',
    'crs' : 'EPSG:32634'
}
eoss_reprojection_url = eoss_collection_root+'{granule_id}?crs={crs}'.format(**reprojectionConfig)


print('Request URL', eoss_reprojection_url)

with request.urlopen(eoss_reprojection_url) as response:
    print('URL for data staged in S3:', response.geturl())

## Spatial Subset

The EOSS API also allows for spatial subsetting of data via specification of a bounding box, as we can see with the collection below.  

In [ ]:
config = {
    'collection_id': 'C1225996408-POCUMULUS',
    'eoss_version': '0.1.0',
    'granule_id' : 'G1226018959-POCUMULUS',
    'west' : '-15.29',
    'south' : '-7.18',
    'east' : '-0.35',
    'north' : '6.83'
}

eoss_spatial_subset_url = harmony_root+'/{collection_id}/eoss/{eoss_version}/items/{granule_id}?bbox={west}%2C{south}%2C{east}%2C{north}'.format(**config)

print('Request URL', eoss_spatial_subset_url)

response = request.urlopen(eoss_spatial_subset_url)
data = response.read()

# Write data to file 
# Change this to save to a directory where you have write permissions
filepath = ('/TBD/harmonySpatialSubset.nc')
file_ = open(filepath, 'wb')
file_.write(data)
file_.close()
